In [1]:
#coding=utf-8
#the first line is necessary to run this code on server

##########################################
# Accounting Factors translated from SAS 
# December 03 2019
# Created by Xinyu LIU
##########################################

import pandas as pd
import numpy as np
import datetime as dt
import wrds
import psycopg2 
import matplotlib.pyplot as plt
from dateutil.relativedelta import *
from pandas.tseries.offsets import *
from pandas.core.frame import DataFrame
from scipy import stats
import datetime
from matplotlib.backends.backend_pdf import PdfPages

###################
# Connect to WRDS #
###################
conn = wrds.Connection(wrds_username='dachxiu')
#make it a constant portal by creating ppass

###################
# Compustat Block #
###################
comp = conn.raw_sql("""
                    select 
                    f.cusip as cnum, c.gvkey, datadate, datadate as
                    datadate_a, fyear, c.cik, sic as sic2, sic, naics, 
                    sale, revt, cogs, xsga, dp, xrd, xad, ib, ebitda, ebit, nopi, spi, pi, txp, 
                    ni, txfed, txfo, txt, xint, 
                    capx, oancf, dvt, ob, gdwlia, gdwlip, gwo, 
                    rect, act, che, ppegt, invt, at, aco, intan, ao, ppent, gdwl, fatb, fatl, 
                    lct, dlc, dltt, lt, dm, dcvt, cshrc, dcpstk, pstk, ap, lco, lo, drc, drlt, txdi,
                    ceq, scstkc, emp, csho, /*addition*/
                    pstkrv, pstkl, txditc, datadate as year, /*market*/
                    abs(prcc_f) as prcc_f, csho*prcc_f as mve_f, /*HXZ*/
                    am, ajex, txdb, seq, dvc, dvp, dp, dvpsx_f, mib, ivao, ivst, sstk, prstkc, 
                    dv, dltis, dltr, dlcch, oibdp, dvpa, tstkp, oiadp, xpp, xacc, re, ppenb, 
                    ppenls, capxv, fopt, wcap
                    from comp.names as c, comp.funda as f
                    where 
                    f.gvkey=c.gvkey
                    /*get consolidated, standardized, industrial format statements*/
                    and f.indfmt='INDL' 
                    and f.datafmt='STD'
                    and f.popsrc='D'
                    and f.consol='C'
                    and datadate >= '01/01/2015'
                    """)

# Due to limited functionality caused by using sql on python, need to manually make up the modifiers work in SAS sql
comp.cnum=comp.cnum.replace(' ','').str.slice(0, 6)
comp.sic2=comp.sic2+'12'
comp.datadate=pd.to_datetime(comp.datadate)
comp.year = comp.datadate.dt.year
comp=comp.dropna(subset=['at','prcc_f','ni'])

# create preferrerd stock
comp['ps']=np.where(comp['pstkrv'].isnull(), comp['pstkl'], comp['pstkrv'])
comp['ps']=np.where(comp['ps'].isnull(),comp['pstk'], comp['ps'])
comp['ps']=np.where(comp['ps'].isnull(),0,comp['ps'])
#manipulate ps data in the sequense of redemption, liquidating and total value, last resolution is 0

comp['txditc']=comp['txditc'].fillna(0)

# create book equity
comp['be']=comp['ceq']+comp['txditc']-comp['ps']
comp['be']=np.where(comp['be']>0,comp['be'],None)
#comp['be']=np.where(comp['be']>0, comp['be'], np.nan)
#Book value of equity equals to Stockholders Equity + Deferred Tax - Preferred Stocks 
#set nan value for book equity that is less than 0

# number of years in Compustat
comp=comp.sort_values(by=['gvkey','datadate']).drop_duplicates()

# number of years in Compustat
comp['count']=comp.groupby(['gvkey']).cumcount()
#Sort DataFrame by column gvkey and datadate
#Mark cumulative number of each gvkey as of that row, starting from 0


###################
# CRSP Block      #
###################
# sql similar to crspmerge macro
crsp_m = conn.raw_sql("""
                      select a.permno, a.permco, a.date, b.ticker, b.ncusip, b.shrcd, b.exchcd, b.siccd,
                      a.prc, a.ret, a.retx, a.shrout, a.vol
                      from crsp.msf as a
                      left join crsp.msenames as b
                      on a.permno=b.permno
                      and b.namedt<=a.date
                      and a.date<=b.nameendt
                      where a.date between '07/01/2016' and '06/30/2018'
                      and b.exchcd between 1 and 3
                      and b.shrcd between 10 and 11
                      """) 
#b.dlprc does not exist

# change variable format to int
crsp_m[['permco','permno','shrcd','exchcd']]=crsp_m[['permco','permno','shrcd','exchcd']].astype(int)

# Line up date to be end of month
crsp_m['date']=pd.to_datetime(crsp_m['date'])
crsp_m['jdate']=crsp_m['date']+MonthEnd(0)
#The 1 in MonthEnd just specifies to move one step forward to the next date that's a month end.

# add delisting return
dlret = conn.raw_sql("""
                     select permno, dlret, dlstdt 
                     from crsp.msedelist
                     """)
#MSEDELIST		CRSP Monthly Stock Event - Delisting
#DLRET 	Num	8	Delisting Return,DLRET is the return of the security after it is delisted. 
#It is calculated by comparing a value after delisting against the price on the security's last trading date. 
#The value after delisting can include a delisting price or the amount from a final distribution.
#DLSTDT 	Num	8	Delisting Date,DLSTDT contains the date (in YYMMDD format) of a security's last price on the current exchange.

#process dlret
dlret.permno=dlret.permno.astype(int)
dlret['dlstdt']=pd.to_datetime(dlret['dlstdt'])
dlret['jdate']=dlret['dlstdt']+MonthEnd(0)

#merge dlret and crsp_m
crsp = pd.merge(crsp_m, dlret, how='left',on=['permno','jdate'])
#crsp and dlret share the same column names: permno and jdate

#process crsp
crsp['dlret']=crsp['dlret'].fillna(0)
crsp['ret']=crsp['ret'].fillna(0)
crsp['retadj']=(1+crsp['ret'])*(1+crsp['dlret'])-1

# calculate market equity
crsp['me']=crsp['prc'].abs()*crsp['shrout']
# Newly added columns in parallel with SAS code, not necessary to be here 
# lag absolute close price, market cap
crsp['prca']=crsp['prc'].abs()
crsp['lprc']=crsp.groupby(['permno','permco'])['prca'].shift(1)
crsp['lme']=crsp.groupby(['permno','permco'])['me'].shift(1)
#market equity equals to price of stock times shares of outstanding

#process crsp
crsp=crsp.drop(['dlret','dlstdt'], axis=1)
crsp=crsp.sort_values(by=['jdate','permco','me']).drop_duplicates()

### Aggregate Market Cap ###
# sum of me across different permno belonging to same permco a given date
crsp_summe = crsp.groupby(['jdate','permco'])['me'].sum().reset_index()
# largest mktcap within a permco/date
crsp_maxme = crsp.groupby(['jdate','permco'])['me'].max().reset_index()
# join by jdate/maxme to find the permno
crsp1=pd.merge(crsp, crsp_maxme, how='inner', on=['jdate','permco','me'])
# drop me column and replace with the sum me
crsp1=crsp1.drop(['me'], axis=1)
# join with sum of me to get the correct market cap info
crsp2=pd.merge(crsp1, crsp_summe, how='inner', on=['jdate','permco'])
# sort by permno and date and also drop duplicates
crsp2=crsp2.sort_values(by=['permno','jdate']).drop_duplicates()
# important to have a duplicate check


# keep December market cap
crsp2['year']=crsp2['jdate'].dt.year
crsp2['month']=crsp2['jdate'].dt.month
decme=crsp2[crsp2['month']==12]
decme=decme[['permno','date','jdate','me','year']].rename(columns={'me':'dec_me'})

### July to June dates
crsp2['ffdate']=crsp2['jdate']+MonthEnd(-6)
crsp2['ffyear']=crsp2['ffdate'].dt.year
crsp2['ffmonth']=crsp2['ffdate'].dt.month
crsp2['1+retx']=1+crsp2['retx']
crsp2=crsp2.sort_values(by=['permno','date'])

# cumret by stock
crsp2['cumretx']=crsp2.groupby(['permno','ffyear'])['1+retx'].cumprod()
#cumprod returns the product of the year in this case, which is the cumulative return as time goes by

# lag cumret
crsp2['lcumretx']=crsp2.groupby(['permno'])['cumretx'].shift(1)

# lag market cap
crsp2['lme']=crsp2.groupby(['permno'])['me'].shift(1)

# if first permno then use me/(1+retx) to replace the missing value
crsp2['count']=crsp2.groupby(['permno']).cumcount()
crsp2['lme']=np.where(crsp2['count']==0, crsp2['me']/crsp2['1+retx'], crsp2['lme'])

# baseline me
mebase=crsp2[crsp2['ffmonth']==1][['permno','ffyear', 'lme']].rename(columns={'lme':'mebase'})

# merge result back together
crsp3=pd.merge(crsp2, mebase, how='left', on=['permno','ffyear'])
crsp3['wt']=np.where(crsp3['ffmonth']==1, crsp3['lme'], crsp3['mebase']*crsp3['lcumretx'])

decme['year']=decme['year']+1
decme=decme[['permno','year','dec_me']]

# Info as of June
crsp3_jun = crsp3[crsp3['month']==6]

crsp_jun = pd.merge(crsp3_jun, decme, how='inner', on=['permno','year'])

# Because I haven't reach the end of the code so will temporarily leave this subslicing open
# crsp_jun=crsp_jun[['permno','date', 'jdate', 'shrcd','exchcd','retadj','me','wt','cumretx','mebase','lme','dec_me']]
crsp_jun=crsp_jun.sort_values(by=['permno','jdate']).drop_duplicates()
crsp_jun=crsp_jun.drop(columns=['count'])

#######################
# CCM Block           #
#######################
ccm=conn.raw_sql("""
                  select gvkey, lpermno as permno, linktype, linkprim, 
                  linkdt, linkenddt
                  from crsp.ccmxpf_linktable
                  where substr(linktype,1,1)='L'
                  and linkprim in ('P', 'C')
                  """)
#CCMXPF_LINKTABLE		CRSP/COMPUSTAT Merged - Link History w/ Used Flag
#lpermno 	Num	8	Historical CRSP PERMNO Link to COMPUSTAT Record
# linktype 	Char	2	Link Type Code,
# Link Type Code is a 2-character code providing additional detail on the usage of the link data available.
# linkprim 	Char	1	Primary Link Marker
# linkdt 	Num	8	First Effective Date of Link
# linkenddt 	Num	8	Last Effective Date of Link

ccm['linkdt']=pd.to_datetime(ccm['linkdt'])
ccm['linkenddt']=pd.to_datetime(ccm['linkenddt'])
# if linkenddt is missing then set to today date
ccm['linkenddt']=ccm['linkenddt'].fillna(pd.to_datetime('today'))
#attention: pd.to.datetime does not convert today(M8[ns]) into format '%Y\%m\%d', need to go with ccm[].dt.date
# if using the code below there will be warning on server
ccm['linkenddt']=ccm['linkenddt'].dt.date

ccm1=pd.merge(comp,ccm,how='left',on=['gvkey'])
ccm1['yearend']=ccm1['datadate']+YearEnd(0)
ccm1['jdate']=ccm1['yearend']+MonthEnd(6)

# set link date bounds
ccm2=ccm1[(ccm1['jdate']>=ccm1['linkdt'])&(ccm1['jdate']<=ccm1['linkenddt'])]
# Subject to further adjustment in the future 
ccm2=ccm2.drop(columns=['datadate_a','linktype','linkdt','linkenddt'])
# ccm2=ccm2[['gvkey','permno','datadate','yearend','jdate','be','op','inv','count']]

# # link comp and crsp
# Note: Different from SAS code, I left merge CCM2 to CRSP_JUN
ccm_jun=pd.merge(crsp_jun, ccm2, how='inner', on=['permno', 'jdate'])
# ccm_data is the SAS parallel
ccm_data=pd.merge(ccm2,crsp_jun, how='left', on=['permno', 'jdate'])
#filtering out prc==nan and dec_me==0
ccm_jun=ccm_jun[ccm_jun.dec_me!=0]
ccm_jun['beme']=ccm_jun['be']*1000/ccm_jun['dec_me']

# drop duplicates
ccm_jun=ccm_jun.sort_values(by=['permno','date']).drop_duplicates()
ccm_jun=ccm_jun.sort_values(by=['gvkey','date']).drop_duplicates()

# Note: Different from SAS, Python count start from zero, will see if I need to add 1 to better serve the need
ccm_jun['count']=ccm_jun.groupby(['gvkey']).cumcount()

# Parallel to the cleaning step for 'dr'
ccm_jun['dr']=np.where(ccm_jun.drc.notna() & ccm_jun.drlt.notna(),ccm_jun.drc+ccm_jun.drlt,None)
ccm_jun['dr']=np.where(ccm_jun.drc.notna() & ccm_jun.drlt.isna(),ccm_jun.drc,ccm_jun['dr'])
ccm_jun['dr']=np.where(ccm_jun.drc.isna() & ccm_jun.drlt.notna(),ccm_jun.drlt,ccm_jun['dr'])
# Parallel to the cleaning step for 'dc'
ccm_jun['dc']=np.where(ccm_jun.dcvt.isna() & ccm_jun.dcpstk.notna() & ccm_jun.pstk.notna() & (ccm_jun.dcpstk>ccm_jun.pstk),\
                       ccm_jun.dcpstk-ccm_jun.pstk,None)
ccm_jun['dc']=np.where(ccm_jun.dcvt.isna() & ccm_jun.dcpstk.notna() & ccm_jun.pstk.isna(),\
                       ccm_jun.dcpstk,ccm_jun['dr'])
ccm_jun['dc']=np.where(ccm_jun.dc.isna(), ccm_jun.dcvt, ccm_jun['dr'])
ccm_jun['xint']=ccm_jun['xint'].fillna(0)
ccm_jun['xsga']=ccm_jun['xsga'].fillna(0)

ccm_jun=ccm_jun.sort_values(by=['permno','date']).drop_duplicates()

Loading library list...
Done


In [ ]:
#######################
# more clean-up and create first pass of variables           #
#######################
#create simple-just annual Compustat variables

ccm_jun['ep']=ccm_jun.ib/ccm_jun.mve_f
ccm_jun['cashpr']=(ccm_jun.mve_f+ccm_jun.dltt-ccm_jun['at'])/ccm_jun.che
ccm_jun['dy']=ccm_jun.dvt/ccm_jun.mve_f
ccm_jun['lev']=ccm_jun['lt']/ccm_jun.mve_f
ccm_jun['sp']=ccm_jun.sale/ccm_jun.mve_f
ccm_jun['roic']=(ccm_jun.ebit-ccm_jun.nopi)/(ccm_jun.ceq+ccm_jun['lt']-ccm_jun.che)
ccm_jun['rd_sale']=ccm_jun.xrd/ccm_jun.sale
ccm_jun['sp']=ccm_jun.sale/ccm_jun.mve_f

#Deleting duplicated columns
# ccm_jun = ccm_jun.loc[:,~ccm_jun.columns.duplicated()]

# treatment for lagged terms
ccm_jun['lagat']=ccm_jun.groupby(['permno'])['at'].shift(1)
ccm_jun['lagcsho']=ccm_jun.groupby(['permno'])['csho'].shift(1)
ccm_jun['laglt']=ccm_jun.groupby(['permno'])['lt'].shift(1)
ccm_jun['lagact']=ccm_jun.groupby(['permno'])['act'].shift(1)
ccm_jun['lagche']=ccm_jun.groupby(['permno'])['che'].shift(1)
ccm_jun['lagdlc']=ccm_jun.groupby(['permno'])['dlc'].shift(1)
ccm_jun['lagtxp']=ccm_jun.groupby(['permno'])['txp'].shift(1)
ccm_jun['laglct']=ccm_jun.groupby(['permno'])['lct'].shift(1)
ccm_jun['laginvt']=ccm_jun.groupby(['permno'])['invt'].shift(1)
ccm_jun['lagemp']=ccm_jun.groupby(['permno'])['emp'].shift(1)
ccm_jun['lagsale']=ccm_jun.groupby(['permno'])['sale'].shift(1)
ccm_jun['lagib']=ccm_jun.groupby(['permno'])['ib'].shift(1)
ccm_jun['lag2at']=ccm_jun.groupby(['permno'])['at'].shift(2)
ccm_jun['lagrect']=ccm_jun.groupby(['permno'])['rect'].shift(1)
ccm_jun['lagcogs']=ccm_jun.groupby(['permno'])['cogs'].shift(1)
ccm_jun['lagxsga']=ccm_jun.groupby(['permno'])['xsga'].shift(1)
ccm_jun['lagppent']=ccm_jun.groupby(['permno'])['ppent'].shift(1)
ccm_jun['lagdp']=ccm_jun.groupby(['permno'])['dp'].shift(1)
ccm_jun['lagxad']=ccm_jun.groupby(['permno'])['xad'].shift(1)
ccm_jun['lagppegt']=ccm_jun.groupby(['permno'])['ppegt'].shift(1)
ccm_jun['lagceq']=ccm_jun.groupby(['permno'])['ceq'].shift(1)
ccm_jun['lagcapx']=ccm_jun.groupby(['permno'])['capx'].shift(1)
ccm_jun['lag2capx']=ccm_jun.groupby(['permno'])['capx'].shift(2)
ccm_jun['laggdwl']=ccm_jun.groupby(['permno'])['gdwl'].shift(1)

ccm_jun['agr']=np.where(ccm_jun['at'].isna() | ccm_jun.lagat.isna(), np.NaN, (ccm_jun.lagat-ccm_jun['at'])/ccm_jun.lagat)
ccm_jun['gma']=ccm_jun.revt-ccm_jun.cogs/ccm_jun.lagat
ccm_jun['chcsho']=ccm_jun.csho/ccm_jun.lagcsho -1
ccm_jun['lgr']=ccm_jun['lt']/ccm_jun.laglt -1
ccm_jun['acc']=(ccm_jun.ib-ccm_jun.oancf)/(ccm_jun['at']+ccm_jun.lagat) -1

ccm_jun['pctacc']=np.where(ccm_jun['ib']==0,(ccm_jun['ib']-ccm_jun['oancf'])/0.01, np.NaN)
ccm_jun['pctacc']=np.where(ccm_jun['oancf'].isna(),(ccm_jun['act']-ccm_jun['lagact']-(ccm_jun['che']-ccm_jun['lagche']))\
                           -(ccm_jun['lct']-ccm_jun['laglct']-(ccm_jun['dlc']-ccm_jun['lagdlc'])\
                            -(ccm_jun['txp']-ccm_jun['lagtxp'])-ccm_jun['dp'])/ccm_jun['ib'].abs(), ccm_jun['pctacc'])
ccm_jun['pctacc']=np.where(ccm_jun['oancf'].isna() & ccm_jun['ib']==0, (ccm_jun['act']-ccm_jun['lagact']-(ccm_jun['che']-ccm_jun['lagche']))\
                           -(ccm_jun['lct']-ccm_jun['laglct']-(ccm_jun['dlc']-ccm_jun['lagdlc'])\
                            -(ccm_jun['txp']-ccm_jun['lagtxp'])-ccm_jun['dp'])/0.01, ccm_jun['pctacc'])

ccm_jun['cfp']= (ccm_jun['ib']-(ccm_jun['act']-ccm_jun['lagact']-(ccm_jun['che']-ccm_jun['lagche'])))\
                -(ccm_jun['lct']-ccm_jun['laglct']-(ccm_jun['dlc']-ccm_jun['lagdlc'])\
                  -(ccm_jun['txp']-ccm_jun['lagtxp'])-ccm_jun['dp'])/ccm_jun['mve_f']
ccm_jun['cfp']=np.where(ccm_jun['oancf'].notna(),ccm_jun['oancf']/ccm_jun['mve_f'], ccm_jun['cfp'])
ccm_jun['absacc']=ccm_jun['acc'].abs()
ccm_jun['chinv']=2*(ccm_jun['invt']-ccm_jun['laginvt'])/(ccm_jun['at']+ccm_jun['lagat'])
ccm_jun['spii']=np.where((ccm_jun['spi']!=0)&ccm_jun['spi'].notna(), 1, 0)

ccm_jun['spi']=2*ccm_jun['spi']/(ccm_jun['at']+ccm_jun['lagat'])
ccm_jun['cf']=2*ccm_jun['oancf']/(ccm_jun['at']+ccm_jun['lagat'])

ccm_jun['cf']=np.where(ccm_jun['oancf'].isna(), (ccm_jun['ib']-(ccm_jun['act']-ccm_jun['lagact']-(ccm_jun['che']-ccm_jun['lagche'])))\
                -(ccm_jun['lct']-ccm_jun['laglct']-(ccm_jun['dlc']-ccm_jun['lagdlc'])\
                  -(ccm_jun['txp']-ccm_jun['lagtxp'])-ccm_jun['dp'])/((ccm_jun['at']+ccm_jun['lagat'])/2),ccm_jun['cf'])  
ccm_jun['hire']=ccm_jun['emp']-ccm_jun['lagemp']/ccm_jun['lagemp']
ccm_jun['hire']=np.where(ccm_jun['emp'].isna() | ccm_jun['lagemp'].isna(), 0, ccm_jun['hire'])

ccm_jun['sgr']=ccm_jun['sale']/ccm_jun['lagsale'] -1
ccm_jun['chpm']=ccm_jun['ib']/ccm_jun['sale']-ccm_jun['lagib']/ccm_jun['lagsale']
ccm_jun['chato']=(ccm_jun['sale']/((ccm_jun['at']+ccm_jun['lagat'])/2)) - (ccm_jun['lagsale']/((ccm_jun['lagat'])+ccm_jun['lag2at'])/2)
ccm_jun['pchsale_pchinvt']=((ccm_jun['sale']-(ccm_jun['lagsale']))/(ccm_jun['lagsale']))-((ccm_jun['invt']-(ccm_jun['laginvt']))/(ccm_jun['laginvt']))
ccm_jun['pchsale_pchrect']=((ccm_jun['sale']-(ccm_jun['lagsale']))/(ccm_jun['lagsale']))-((ccm_jun['rect']-(ccm_jun['lagrect']))/(ccm_jun['lagrect']))
ccm_jun['pchgm_pchsale']=(((ccm_jun['sale']-ccm_jun['cogs'])-((ccm_jun['lagsale'])-(ccm_jun['lagcogs'])))/((ccm_jun['lagsale'])-(ccm_jun['lagcogs'])))-((ccm_jun['sale']-(ccm_jun['lagsale']))/(ccm_jun['lagsale']))
ccm_jun['pchsale_pchxsga']=((ccm_jun['sale']-(ccm_jun['lagsale']))/(ccm_jun['lagsale']) )-((ccm_jun['xsga']\
        -(ccm_jun['lagxsga'])) /(ccm_jun['lagxsga']) )
ccm_jun['depr']=ccm_jun['dp']/ccm_jun['ppent']
ccm_jun['pchdepr']=((ccm_jun['dp']/ccm_jun['ppent'])-((ccm_jun['lagdp'])/(ccm_jun['lagppent'])))/((ccm_jun['lagdp'])/(ccm_jun['lagppent']))
ccm_jun['chadv']=np.log(1+ccm_jun['xad'])-np.log((1+(ccm_jun['lagxad'])))
ccm_jun['invest']=((ccm_jun['ppegt']-(ccm_jun['lagppegt'])) +  (ccm_jun['invt']-(ccm_jun['laginvt'])) ) / (ccm_jun['lagat'])
ccm_jun['invest']=np.where(ccm_jun['ppegt'].isna(), ((ccm_jun['ppent']-(ccm_jun['lagppent'])) +  (ccm_jun['invt']-(ccm_jun['laginvt'])) ) / (ccm_jun['lagat']), ccm_jun['invest'])
ccm_jun['egr']=((ccm_jun['ceq']-(ccm_jun['lagceq']))/(ccm_jun['lagceq']) )

# Note here instead of using count>=2, I use 1 to stay in line iwth python cumcount
# Also starting from here I'll keep the SAS in the notes for comparison and debug
    # 	if missing(capx) and count>=2 then
    # 		capx=ppent-lag(ppent);
    # 	pchcapx=(capx-lag(capx))/lag(capx);
    # 	grcapx=(capx-lag2(capx))/lag2(capx);
    # 	grGW=(gdwl-lag(gdwl))/lag(gdwl);
ccm_jun['capx']=np.where(ccm_jun['capx'].isna() & ccm_jun['count']>=1,ccm_jun['ppent']-(ccm_jun['lagppent']), ccm_jun['capx'])
ccm_jun['pchcapx']=ccm_jun['capx']-ccm_jun['lagcapx']/ccm_jun['lagcapx']
ccm_jun['grcapx']=ccm_jun['capx']-ccm_jun['lag2capx']/ccm_jun['lag2capx']
ccm_jun['grGW']=ccm_jun['gdwl']-ccm_jun['laggdwl']/ccm_jun['laggdwl']
    # 	if missing(gdwl) or gdwl=0 then
    # 		grGW=0;
    # 	if gdwl ne 0 and not missing(gdwl) and missing(grGW) then
    # 		grGW=1;
ccm_jun['grGW']=np.where(ccm_jun['gdwl'].isna() | ccm_jun['gdwl']==0, 0, ccm_jun['grGW'])  
ccm_jun['grGW']=np.where(ccm_jun['gdwl'].notna() & ccm_jun['gdwl']!=0 & ccm_jun['grGW'].isna(), 1, ccm_jun['grGW']) 
    # 	if (not missing(gdwlia) and gdwlia ne 0) or (not missing(gdwlip) and gdwlip ne 
    # 		0) or (not missing(gwo) and gwo ne 0) then
    # 			woGW=1;
ccm_jun['woGW']=np.where((ccm_jun['gdwlia'].notna()&ccm_jun['gdwlia']!=0)|(ccm_jun['gdwlip'].notna()&(ccm_jun['gdwlip']!=0))|\
                                                                          (ccm_jun['gwo'].notna()&ccm_jun['gwo']!=0) , 1, 0)
    #	tang=(che+rect*0.715+invt*0.547+ppent*0.535)/at;
ccm_jun['tang']=(ccm_jun['che']+ccm_jun['rect']*0.715+ccm_jun['invt']*0.547+ccm_jun['ppent']*0.535)/ccm_jun['at']